# Dataset Inspection

In [ ]:
from __future__ import print_function, division

from pathlib2 import Path

import numpy as np
import rospy

from vgn import from_voxel_coordinates
from vgn.dataset import Dataset
from vgn.grasp import Grasp
from vgn.utils.transform import Rotation, Transform
from vgn_ros import vis

In [ ]:
dataset_dir = Path("../data/datasets/blocks")

finger_depth = 0.05
size = 6.0 * finger_depth
voxel_size = size / 40.0

In [ ]:
# initialize a ros node to communicate with rviz
rospy.init_node("dataset_inspection")

In [ ]:
# load the dataset
dataset = Dataset(dataset_dir)

## Stats

In [ ]:
df = dataset.df

positives = df[df["label"] == 1]
negatives = df[df["label"] == 0]

print("Number of samples:", len(df.index))
print("Number of positives:", len(positives.index))
print("Number of negatives:", len(negatives.index))

## Visualize

In [ ]:
i = np.random.choice(len(dataset))
tsdf, (label, rotations, width), index = dataset[i]
grasp = Grasp(Transform(Rotation.from_quat(rotations[0]), index), width)

grasp = from_voxel_coordinates(grasp, voxel_size)

vis.clear()
vis.workspace(size)
vis.tsdf(tsdf.squeeze(), voxel_size)
vis.grasps([grasp], [float(label)], finger_depth)